## Read Data
train data: [training.json](../../../data/groundtruth_dataset/Binary_PIP_Classifier/training.json)

test data: [testing.json](../../../data/groundtruth_dataset/Binary_PIP_Classifier/testing.json)

In [ ]:
import json
import pandas as pd
def read_data(filepath):
    data = []
    with open(filepath,'r') as f:
        for line in f.readlines():
            data.append(json.loads(line))
    return data
#TODO: 
train_data_path = ''
test_data_path = ''

train_data = read_data(train_data_path)
test_data = read_data(test_data_path)
train_df = pd.DataFrame(train_data)
eval_df = pd.DataFrame(test_data)

In [5]:
print("Train Set", len(train_data))
print("Test Set", len(test_data))

Train Set 10544
Test Set 2637


In [9]:
#language-wise and category-wise distribution of groundtruth dataset
def distribution(data_list):
    balanced_language = {}
    for each in data_list:
        lang = each['language']
        if lang not in balanced_language:
            balanced_language[lang] = {}
            balanced_language[lang]['positive'] = 0
            balanced_language[lang]['negative'] = 0
            balanced_language[lang]['total'] = 0
        if int(each['labels']):
            balanced_language[lang]['positive'] += 1
        else:
            balanced_language[lang]['negative'] += 1
        balanced_language[lang]['total'] += 1
    balanced_language = dict(sorted(balanced_language.items(), key=lambda x: x[1]['total'], reverse=True))
    balanced_language = pd.DataFrame(balanced_language).T
    print('language-wise distribution of train data: ')
    print(balanced_language)
    
    balanced_category = {}
    for each in data_list:
        if int(each['labels']):
            if each['subcategory'] not in balanced_category:
                balanced_category[each['subcategory']] = 0
            balanced_category[each['subcategory']] += 1
    balanced_category = dict(sorted(balanced_category.items(), key=lambda x: x[1], reverse=True))
    balanced_category = pd.DataFrame(balanced_category.items(), columns=['Key', 'Value'])
    print('\ncategory-wise distribution of train data: ')
    print(balanced_category)
    
distribution(train_data)

language-wise distribution of train data: 
    positive  negative  total
en      2024      2257   4281
zh      3436       301   3737
ja       463       471    934
it       142       137    279
th       141       134    275
de       112       128    240
es       119       112    231
ru        99        95    194
ko        96        93    189
fr        84       100    184

category-wise distribution of train data: 
              Key  Value
0            porn   3002
1            drug    772
2        Gambling    641
3   money-laundry    599
4    data_leakage    591
5    Crowdturfing    342
6      harassment    271
7          weapon    162
8   fake_document    140
9       surrogacy    104
10         Others     92


## Binary Text Classifier
### Training
model name: bert-base-multilingual-uncased

In [8]:
#training
from simpletransformers.classification import ClassificationModel

EPOCH_NUM = 14
args={'num_train_epochs':EPOCH_NUM, 'output_dir': "outputs"}
text_model = ClassificationModel("bert", "bert-base-multilingual-uncased", args=args)

train_df_text = train_df
train_df_text['labels'] = train_df_text['labels'].replace(['0','1'],[0,1]) 

text_model.train_model(train_df_text, output_dir=f"output_text_train")

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model 

Model created and data ready, starting training...


  0%|          | 0/10674 [00:00<?, ?it/s]

Epoch:   0%|          | 0/20 [00:00<?, ?it/s]

Running Epoch 0 of 20:   0%|          | 0/1335 [00:00<?, ?it/s]

...

Running Epoch 16 of 20:   0%|          | 0/1335 [00:00<?, ?it/s]

Running Epoch 17 of 20:   0%|          | 0/1335 [00:00<?, ?it/s]

Running Epoch 18 of 20:   0%|          | 0/1335 [00:00<?, ?it/s]

Running Epoch 19 of 20:   0%|          | 0/1335 [00:00<?, ?it/s]

(26700, 0.18389842666936723)

### Evaluation

In [4]:
#evaluation
import os, re
from simpletransformers.classification import ClassificationModel

eval_df_text = eval_df
eval_df_text['labels'] = eval_df_text['labels'].replace(['0','1'],[0,1]) 
model_name = 'outputs/checkpoint-18452-epoch-14'
text_model = ClassificationModel("bert", model_name)
result_text, model_text_outputs, wrong_predictions = text_model.eval_model(eval_df_text)

wrong_texts = [each.text_a for each in wrong_predictions]
result_text['precision'] = result_text['tp'] / (result_text['tp'] + result_text['fp'])
result_text['recall'] = result_text['tp'] / (result_text['tp'] + result_text['fn'])
result_text['F1_score'] = 2 * result_text['precision'] * result_text['recall'] / (result_text['precision'] + result_text['recall'])
print(pd.DataFrame(result_text.items(), columns=['Key', 'Value']))

language_wise_performance = {}
for each in test_data:
    lang = each['language']
    text = each['text']
    if lang not in language_wise_performance:
        language_wise_performance[lang] = {}
        language_wise_performance[lang]['TP'] = 0
        language_wise_performance[lang]['FP'] = 0
        language_wise_performance[lang]['TN'] = 0
        language_wise_performance[lang]['FN'] = 0
    if int(each['labels']) == 1 and text not in wrong_texts:
        language_wise_performance[lang]['TP'] += 1
    elif int(each['labels']) == 0 and text not in wrong_texts:
        language_wise_performance[lang]['TN'] += 1 
    elif int(each['labels']) == 0 and text in wrong_texts:
        language_wise_performance[lang]['FP'] += 1
    elif int(each['labels']) == 1 and text in wrong_texts:
        language_wise_performance[lang]['FN'] += 1
for lang in language_wise_performance:
    language_wise_performance[lang]['count'] = language_wise_performance[lang]['TP'] + language_wise_performance[lang]['FP'] + language_wise_performance[lang]['TN'] + language_wise_performance[lang]['FN']
    language_wise_performance[lang]['precision'] = language_wise_performance[lang]['TP'] / (language_wise_performance[lang]['TP'] + language_wise_performance[lang]['FP'])
    language_wise_performance[lang]['recall'] = language_wise_performance[lang]['TP'] / (language_wise_performance[lang]['TP'] + language_wise_performance[lang]['FN'])
    language_wise_performance[lang]['F1_score'] = 2 * language_wise_performance[lang]['precision'] * language_wise_performance[lang]['recall'] / (language_wise_performance[lang]['precision'] + language_wise_performance[lang]['recall'])
language_wise_performance = dict(sorted(language_wise_performance.items(), key=lambda x: x[1]['count'], reverse=True))
language_wise_performance = dict(list(language_wise_performance.items())[:10])
print('\nlanguage-wise performance of test set:')
print(pd.DataFrame(language_wise_performance).T)

category_wise_performance = {}
for each in test_data:
    if 'subcategory' not in each:
        continue
    cat = each['subcategory']
    text = each['text']
    if cat not in category_wise_performance:
        category_wise_performance[cat] = {}
        category_wise_performance[cat]['TP'] = 0
        category_wise_performance[cat]['FN'] = 0
    if text in wrong_texts:
        category_wise_performance[cat]['FN'] += 1
    else:
        category_wise_performance[cat]['TP'] += 1
for cat in category_wise_performance:
    category_wise_performance[cat]['count'] = category_wise_performance[cat]['TP'] + category_wise_performance[cat]['FN']
    category_wise_performance[cat]['recall'] = category_wise_performance[cat]['TP'] / category_wise_performance[cat]['count']
category_wise_performance = dict(sorted(category_wise_performance.items(), key=lambda x: x[1]['count'], reverse=True))
print('\ncategory-wise performance of test set:')
print(pd.DataFrame(category_wise_performance).T)

  0%|          | 0/2637 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/330 [00:00<?, ?it/s]

          Key        Value
0         mcc     0.907409
1          tp  1625.000000
2          tn   899.000000
3          fp    46.000000
4          fn    67.000000
5       auroc     0.973540
6       auprc     0.975187
7   eval_loss     0.228150
8   precision     0.972472
9      recall     0.960402
10   F1_score     0.966399

language-wise performance of test set:
       TP    FP     TN    FN   count  precision    recall  F1_score
en  474.0  16.0  568.0  19.0  1077.0   0.967347  0.961460  0.964395
zh  854.0  19.0   49.0  18.0   940.0   0.978236  0.979358  0.978797
ja  115.0   5.0  111.0  11.0   242.0   0.958333  0.912698  0.934959
th   28.0   4.0   31.0  10.0    73.0   0.875000  0.736842  0.800000
de   42.0   0.0   28.0   0.0    70.0   1.000000  1.000000  1.000000
it   31.0   0.0   31.0   0.0    62.0   1.000000  1.000000  1.000000
es   24.0   1.0   26.0   2.0    53.0   0.960000  0.923077  0.941176
ru   21.0   0.0   23.0   1.0    45.0   1.000000  0.954545  0.976744
ko   16.0   1.0   19.0  